In [2]:
import os
import requests
from dotenv import load_dotenv

load_dotenv()

# Configuration
CLIENT_ID = os.getenv('LINKEDIN_CLIENT_ID')
CLIENT_SECRET = os.getenv('LINKEDIN_CLIENT_SECRET')
REDIRECT_URI = os.getenv('REDIRECT_URI')
TOKEN_FILE = 'linkedin_token.txt'

# LinkedIn API endpoints
AUTH_URL = 'https://www.linkedin.com/oauth/v2/authorization'
TOKEN_URL = 'https://www.linkedin.com/oauth/v2/accessToken'
POST_URL = 'https://api.linkedin.com/v2/ugcPosts'

def get_auth_code():
    params = {
        'response_type': 'code',
        'client_id': CLIENT_ID,
        'redirect_uri': REDIRECT_URI,
        'scope': 'w_member_social'
    }
    print(f"Authorize here: {AUTH_URL}?{'&'.join([f'{k}={v}' for k, v in params.items()])}")
    return input("Enter authorization code: ")

def get_access_token(code):
    data = {
        'grant_type': 'authorization_code',
        'code': code,
        'redirect_uri': REDIRECT_URI,
        'client_id': CLIENT_ID,
        'client_secret': CLIENT_SECRET
    }
    response = requests.post(TOKEN_URL, data=data)
    response.raise_for_status()
    token_data = response.json()
    with open(TOKEN_FILE, 'w') as f:
        f.write(token_data['access_token'])
    return token_data['access_token']

def load_token():
    if not os.path.exists(TOKEN_FILE):
        code = get_auth_code()
        return get_access_token(code)
    with open(TOKEN_FILE, 'r') as f:
        return f.read()

def create_post(text):
    access_token = load_token()
    headers = {
        'Authorization': f'Bearer {access_token}',
        'Content-Type': 'application/json',
        'X-Restli-Protocol-Version': '2.0.0'
    }
    
    # Update with your LinkedIn URN (format: urn:li:person:{id})
    author_urn = 'urn:li:person:YOUR_MEMBER_ID'  
    
    post_data = {
        "author": author_urn,
        "lifecycleState": "PUBLISHED",
        "specificContent": {
            "com.linkedin.ugc.ShareContent": {
                "shareCommentary": {
                    "text": text
                },
                "shareMediaCategory": "NONE"
            }
        },
        "visibility": {
            "com.linkedin.ugc.MemberNetworkVisibility": "PUBLIC"
        }
    }
    
    response = requests.post(POST_URL, headers=headers, json=post_data)
    return response.json()

if __name__ == '__main__':
    post_text = "Hello LinkedIn! This is my first post via API. #automation"
    result = create_post(post_text)
    print("Post created:", result)

Authorize here: https://www.linkedin.com/oauth/v2/authorization?response_type=code&client_id=your_client_id_here&redirect_uri=None&scope=w_member_social


HTTPError: 401 Client Error: Unauthorized for url: https://www.linkedin.com/oauth/v2/accessToken